#1

In [ ]:
import transformers
print(transformers.__version__)


#2

In [ ]:
from transformers import pipeline
sentiment = pipeline('sentiment-analysis')


In [ ]:
sentiment.model

In [ ]:
print(sentiment(["I like Olympic games as it's very exciting."]))
print(sentiment(["I'm against to hold Olympic games in Tokyo in terms of preventing the covid19 to be spread."]))


# 3

In [ ]:
from transformers import pipeline

qa = pipeline("question-answering")

# 여러 줄로 이루어진 문장을 인용할 경우 """를 인용부호로 사용
olympic_wiki_text = """
The 2020 Summer Olympics (Japanese: 2020年夏季オリンピック, Hepburn: Nisen Nijū-nen Kaki Orinpikku), officially the Games of the XXXII Olympiad (第三十二回オリンピック競技大会, Dai Sanjūni-kai Orinpikku Kyōgi Taikai) and branded as Tokyo 2020 (東京2020), is an ongoing international multi-sport event being held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July.

Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.[3] Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 in March 2020 as a result of the COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled).[4] However, the event retains the Tokyo 2020 name for marketing and branding purposes.[5] It is being held largely behind closed doors with no public spectators permitted due to the declaration of a state of emergency.[b] The Summer Paralympics will be held between 24 August and 5 September 2021, 16 days after the completion of the Olympics.[6]

The 2020 Games are the fourth Olympic Games to be held in Japan, following the Tokyo 1964 (Summer), Sapporo 1972 (Winter), and Nagano 1998 (Winter) games.[c] Tokyo is the first city in Asia to hold the Summer Games twice. The 2020 Games are the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea, and preceding the 2022 Winter Olympics in Beijing, China.

The 2020 Games introduced new competitions and re-introduced competitions that once were held but were subsequently removed. New ones include 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allow the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee are baseball and softball, karate, sport climbing, surfing, and skateboarding, the last four of which make their Olympic debuts.[7]

Bermuda, the Philippines, and Qatar won their first-ever Olympic gold medals.[8][9][10] San Marino, Turkmenistan, and Burkina Faso won their first-ever Olympic medals.[11][12][13]

"""
print(qa(question="What caused Tokyo Olympic postponed?", context=olympic_wiki_text))


In [ ]:
qa.model

# Preparation

In [ ]:
import torch
print(f"cuda support: {torch.cuda.is_available()}")
print(f"mac metal support: {torch.backends.mps.is_available()}")

# 4

In [ ]:
from torchtext.datasets import IMDB

train_iter = IMDB(split='train')
test_iter = IMDB(split='test')

# from torchtext.datasets import IMDB
# from torchtext.data import Field, LabelField
# from torchtext.data.utils import get_tokenizer

# tokenizer = get_tokenizer('basic_english')

# TEXT = Field(sequential=True, tokenize=tokenizer, lower=True)
# LABEL = LabelField(dtype=torch.float)

# train_iter, test_iter = IMDB.splits(TEXT, LABEL)

# type(train_iter)

In [ ]:
# 런타임 약 30초 소요
# 출력 결과를 고정하기 위해 random.seed 도입
import random
random.seed(6)

# train_iter를 리스트 타입으로 변경
train_lists = list(train_iter)
test_lists = list(test_iter)

# 각기 1000개씩 랜덤 샘플링
train_lists_small = random.sample(train_lists, 1000)
test_lists_small = random.sample(test_lists, 1000)

# 각 변수에 담긴 인덱스 0에 해당하는 원소, 즉 첫번째 원소 출력
print(train_lists_small[0])
print(test_lists_small[0])


In [ ]:
# train_texts와 train_labels라는 컨테이너 생성
# 아래 반복분에서 생성된 결과를 담는 그릇으로서 역할 수행
train_texts = []
train_labels = []

# for 반복문
# train_lists_small에 담긴 튜플 쌍 원소를 변수명 label과 text를 부여하여 순서대로 추출
for label, text in train_lists_small:
  # IMDB 데이터의 기존 레이블 2를 1로 변경, 기존 레이블 1을 0으로 변경
  train_labels.append(1 if label == 2 else 0)
  train_texts.append(text)

# text_texts와 test_labels라는 컨테이너 생성
test_texts = []
test_labels = []

# for 반복문
for label, text in test_lists_small:
  # IMDB 데이터의 기존 레이블 2를 1로 변경, 기존 레이블 1을 0으로 변경
  test_labels.append(1 if label == 2 else 0)
  test_texts.append(text)

# 각 변수에 담긴 인덱스 0에 해당하는 원소, 즉 첫번째 원소 출력
print(train_texts[0])
print(train_labels[0])
print(test_texts[0])
print(test_labels[0])


#6

In [ ]:
# train_test_split 결과를 고정하기 위해 random_state 지정
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2, random_state=3)
print(len(train_texts))
print(len(train_labels))
print(len(val_texts))
print(len(val_labels))
print(len(test_texts))
print(train_labels[0])


#7

In [ ]:
# distilbert-base-uncased 모델에서 토크나이저 불러오기
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
# 토크나이징을 통한 인코딩
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

print(train_texts)
print(len(train_texts))
# print(train_encodings.keys())
# print(tokenizer.decode(train_encodings['input_ids'][0]))
# print(tokenizer.decode(train_encodings['attention_mask'][0]))
# print(tokenizer.decode(train_encodings['input_ids'][1]))
# print(tokenizer.decode(train_encodings['attention_mask'][1]))
# print(len(train_encodings['input_ids'][0]))
# print(len(val_encodings['input_ids'][0]))
# print(len(test_encodings['input_ids'][0])) # all 512 why?

# # 0번째 입력문(텍스트)의 5번째 토큰까지의 input_ids 출력
print(train_encodings["input_ids"][0][:5])

# 위의 결과를 디코딩하여 출력
print(tokenizer.decode(train_encodings["input_ids"][0][:5]))


# 8

In [ ]:
import torch

# Dataset 클래스를 상속하는 IMDBDataset 클래스를 정의
class IMDbDataset(torch.utils.data.Dataset):

    # 생성자 __init__()
    # 자신을 가르키는 매개변수 self 포함
    # 변수를 저장하기 위해 self.변수명을 사용
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    # 자신을 가르키는 매개변수 self 포함
    def __getitem__(self, idx):
        # self.encoding에 담긴 키(key)와 키값(value)을 items()로 추출
        # 이 값을 key와 val 변수에 담아 새로운 키(key)와 키값(torch.tensor(val[idx]))를 갖는 딕셔너리 생성
        # 딕셔너리는 {"key1:value1", "key2:value2", ,,,} 형태를 지닌 파이썬 데이터 구조
        # val[idx]에 담긴 데이터를 torch.tensor()를 통해 파이토치 텐서로 변환
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        # self.lables[idx]에 담긴 데이터를 torch.tensor()를 통해 파이토치 텐서 변환
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    # 자신을 가르키는 매개변수 self 포함
    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

# print(next(iter(train_dataset.encodings.items())))
for key, val in train_encodings.items():
    print("...")
    print(key, val)



In [ ]:
for i in train_dataset:
  print(i)
  break


# 9

In [ ]:
from transformers import DistilBertForSequenceClassification

# distilbert 모델 불러오기
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
model


#10

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # 출력 디렉토리 경로
    # num_train_epochs=8,              # 학습 에포크 수
    num_train_epochs=6,              # 학습 에포크 수
    per_device_train_batch_size=16,  # 학습시 (디바이스 별) 미니 배치 수
    per_device_eval_batch_size=64,   # 평가시 (디바이스 별) 미니 배치 수
    warmup_steps=500,                # 학습률 스케줄링용 웜업 스텝 수

    weight_decay=0.01,               # 가중치 감쇄 강도
    logging_dir='./logs',            # 로그 디렉토리 경로
    logging_steps=10,
)


# 11

In [ ]:
import torch

# GPU 사용이 가능한 경우 텐서 타입 데이터를 GPU로 전달
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
model = model.to(device)
print(model.device)


In [ ]:
!nvidia-smi

# 12

In [ ]:
# 파인튜닝 이전에 세 입력 문장 극성 판별
# 이는 한국어판을 취해 추가된 코드임
input_tokens = tokenizer(["I feel fantastic", "My life is going something wrong", "I have not figured out what the chosen title has to do with the movie."], truncation=True, padding=True)

# 입력 문장 토크나이징 결과(input_tokens)에 담긴 input_ids를 모델에 투입
# 그리고 모델 출력 결과를 GPU로 전송하며 값은 변수 outputs에 저장
outputs = model(torch.tensor(input_tokens['input_ids']).to(device))

# 레이블 딕셔너리 생성
label_dict = {1:'positive', 0:'negative'}


# outputs 변수에 담긴 logits 값을 행 단위, 즉 입력 문장 단위로 가장 큰 값 위치(인덱스) 추출
# 그 결과값(인덱스)을 cpu로 넘기고 넘파이 타입으로 변경 후, 인덱스에 매칭되는 레이블 출력
print([label_dict[i] for i in torch.argmax(outputs['logits'], axis=1).cpu().numpy()])

# print(outputs['logits'])

In [ ]:
# 런타임 약 4 혹은 5분 소요
# 딥러닝 모델의 특성상 결과가 매번 약간 다르게 나올 수 있슴
from transformers import Trainer
trainer = Trainer(
    model=model,                      # 사전학습 모델 인스턴스화
    args=training_args,               # TrainingArguments에 정의한 하이퍼 파라미터값 가져오기
    train_dataset=train_dataset,      # 학습 데이터세트
    eval_dataset=val_dataset          # evaluation 데이터세트
)

trainer.train()


In [ ]:
# Trainer.train()에 의한 파인튜팅 이후 세 입력 문장 극성 판별
input_tokens = tokenizer(["I feel fantastic", "My life is going something wrong", "I have not figured out what the chosen title has to do with the movie."], truncation=True, padding=True)
outputs = model(torch.tensor(input_tokens['input_ids']).to(device))
# 영어 원문의 1과 0값과 달리 일반적인 용례에 따라 변경
label_dict = {1:'positive', 0:'negative'}
print([label_dict[i] for i in torch.argmax(outputs['logits'], axis=1).cpu().numpy()])

In [ ]:
# 문제14에서 언급한 Trainer.train( ) 방식의 성과지표로서의 정확도를 여기서 구함
# 모델 14의 주 10번 참조
# 모델을 eval 모드로 전환
model.eval()

# 문제 14 코딩의 변수 l과 혼동을 피하기 위해 컨테이너 리스트 l2 생성
l2 = []

# 반복 루프
for test_text in test_texts:

  # 토크나이징을 통한 인코딩
  input_tokens = tokenizer([test_text], truncation=True, padding=True)

  # 모델을 사용한 추론
  outputs = model(torch.tensor(input_tokens['input_ids']).to(device))

  # outputs에 담긴 logits값을 기준으로 행 단위로, 즉 입력 문장 단위로 가장 큰 logits값의 인덱스를 출력 후
  # item( )을 사용하여 결과물 텐서를 CPU로 전송
  # 이렇게 처리된 값을 입력문장별로 append( )를 통해 컨테이너 리스트 l에 하나씩 저장
  l2.append(torch.argmax(outputs['logits'], axis=1).item())

# 변수 correct_cnt 초기값을 0으로 설정
correct_cnt = 0

# 리스트 l과 test_labels를 zip( )을 통해 쌍(pair)으로 묶은 후 각기 pred와 ans라는 변수로 추출
# zip()안의 데이터 쌍이 소진될 때까지 if문을 반복하여 계속 실행
for pred, ans in zip(l2, test_labels):
  if pred == ans:
    correct_cnt += 1

# 정확도(accuracy) 계산
# 딥러닝 모델의 특성상 실행시마다 성과지표가 약간 다르게 산출될 수 있음
# correct_cnt는 pred와 ans가 일치하는 건수의 총합
# test_labels는 레이블 개수의 총합, 즉 입력 문장 전체 건수
print(correct_cnt/len(test_labels))

# 13

In [ ]:
# test_inference 함수 생성
def test_inference(model, tokenizer):
  input_tokens = tokenizer(["I feel fantastic", "My life is going something wrong", "I have not figured out what the chosen title has to do with the movie."], truncation=True, padding=True)
  outputs = model(torch.tensor(input_tokens['input_ids']).to(device))
  # 영어 원문의 1과 0값 배정고 달리 일반적인 용례에 따라 변경
  label_dict = {1:'positive', 0:'negative'}
  return [label_dict[i] for i in torch.argmax(outputs['logits'], axis=1).cpu().numpy()]

In [ ]:
# 런타임 5분 30초 소요
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW
from transformers import DistilBertTokenizerFast

#１） 사전학습 모델과 토크나이저 불러오기
#    그리고 모델 실행결과에 to(devic) 코드 추가(가능한 경우 결과를 GPU에 전달)
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(device)

# 파인튜닝 이전 모델을 사용하여 test_inference 함수 실행
print(test_inference(model, tokenizer))

#２）DataLoader 인스턴스화
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

#３）최적화 함수 정의
optim = AdamW(model.parameters(), lr=5e-5)

#４）모델을 학습(train) 모드로 전환
#   이는 드롭아웃 및 배치 정규화에 영향을 미침
model.train()

losses = []

#５）에포크 횟수(epochs)만큼 루프 반복
for epoch in range(8):
    print(f'epoch:{epoch}')
    for batch in train_loader:
        #６）최적화 함수의 기울기(그래디언트) 초기화
        optim.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        #７） 모델을 사용한 추론
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        #８） 손실 계산
        loss = outputs[0]
        losses.append(loss)
        
        #９） 오차역전파
        loss.backward()
        #１０） 가중치(weight) 업데이트
        optim.step()

# 모델을 eval 모드로 전환
model.eval()

# eval 모드를 사용하여 test_inference 함수 실행
print(test_inference(model, tokenizer))


In [ ]:
# 다음은 에러가 발생하는 코드임에 주의
# 이를 수정하기 위해 몇 개의 코드를 아래에 추가함
import matplotlib.pyplot as plt
plt.plot(losses)
plt.show()


In [ ]:
print(losses)
type(losses)

In [ ]:
# 파이토치에서 item( )은 텐서에서 값을 파이썬 숫자로 추출하여 CPU로 전달
new_losses = [i.item() for i in losses]

# 처음 5개 값 출력
new_losses[:5]

In [ ]:
# 마지막 5개 값 출력
new_losses[-5:]

In [ ]:
# matplotlib 라이브러리의 pyplot 모듈을 약칭 plt로 import
import matplotlib.pyplot as plt
plt.plot(new_losses)
plt.show();

#14

In [ ]:
# 모델을 eval 모드로 전환
model.eval()

# 변수 l이라는 컨테이너 리스트 생성
l = []

print(len(test_texts))
print(test_texts[0])
# 반복 루프
for test_text in test_texts:
  print(f"test_text: {test_text}")

  # 토크나이징을 통한 인코딩
  input_tokens = tokenizer([test_text], truncation=True, padding=True)

  # 모델을 사용한 추론
  outputs = model(torch.tensor(input_tokens['input_ids']).to(device))

  # outputs에 담긴 logits값을 기준으로 행 단위로, 즉 입력 문장 단위로 가장 큰 logits값의 인덱스를 출력 후
  # item( )을 사용하여 결과물 텐서 값을 추출하고 CPU로 전송
  # 이렇게 처리된 값을 입력문장별로 append( )를 통해 컨테이너 리스트 l에 하나씩 저장
  l.append(torch.argmax(outputs['logits'], axis=1).item())

print(f"l: {l}")

# 변수 correct_cnt 초기값을 0으로 설정
correct_cnt = 0

# 리스트 l과 test_labels를 zip( )을 통해 쌍(pair)으로 묶은 후 각기 pred와 ans라는 변수로 추출
# zip()안의 데이터 쌍이 소진될 때까지 if문을 반복하여 계속 실행
for pred, ans in zip(l, test_labels):
  if pred == ans:
    # if 조건문이 참인 경우 correct_cnt값을 1씩 증가
    correct_cnt += 1

# 정확도(accuracy) 계산
# correct_cnt는 pred와 ans가 일치하는 건수의 총합
# len(test_labels)는 레이블 개수의 총합, 즉 입력 문장 전체 건수
print(correct_cnt/len(test_labels))


# 15

In [ ]:
# 각 변수의 초기값을 0으로 설정
tp = 0
tn = 0
fp = 0
fn = 0

# 리스트 l과 test_labels를 zip( )을 통해 쌍(pair)으로 묶은 후 각기 pred와 ans라는 변수로 추출
# zip()안의 데이터 쌍이 소진될 때까지 반복문을 실행
for pred, ans in zip(l, test_labels):
  print(f"pred: {pred}, ans: {ans}")

  # tp 계산
  if pred == ans:
    if pred ==1:
      tp += 1
    else:
      tn += 1

  # fn 계산
  elif pred == 0:
    fn += 1

  # fp 계산
  elif pred == 1:
    fp +=1


In [ ]:
recall = tp/(tp+fn)
print(recall)


In [ ]:
precision = tp/(tp+fp)
print(precision)


In [ ]:
(2 * precision * recall) / (precision + recall)

#16

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(test_labels, l))
